
# <center> Dataton Bancolombia 2022: propuesta y diseño de un recomendador de noticias <center/>


El siguiente trabajo fue elaborado por Samuel Vasco G.¹, J. Nicolay Ruiz A.¹.

¹Instituto de Física de la Universidad de Antioquia

## Cargar dataset (3 archivos csv)

In [ ]:
import pandas as pd 
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

La lectura y procesado de los datos se resume de la siguiente manera:

Primeramente se cargan los tres datasets (_`cliente.csv`_, _`noticias.csv`_, _`clientes_noticias.csv`_).

Se añade el nombre y subsector de cada cliente al dataset que contiene los clientes y noticias (_`clientes_noticias.csv`_). Se elimina la información de los intervalos de fechas porque no se usarán. Se añade el título de la noticia y el contenido de la misma al dataset que se está construyendo. Posteriormente se ordena de acuerdo a la identificación del cliente. Se eliminan todos los caractéres que sean diferentes a letras, además de los meses y días. Adicionalmente, se conservara todo minúsculas.

Finalmente, por motivos de practicidada, se guarda este dataset para ser usado en el presente trabajo.

In [ ]:
clientes = pd.read_csv('/content/drive/MyDrive/Bancolombia_Dataton_2022/clientes.csv')
noticias = pd.read_csv('/content/drive/MyDrive/Bancolombia_Dataton_2022/noticias.csv')
users_news = pd.read_csv('/content/drive/MyDrive/Bancolombia_Dataton_2022/clientes_noticias.csv')

users_news = pd.merge(users_news, clientes[['nit', 'nombre', 'subsec']], on='nit')
users_news = users_news.drop(columns=['news_init_date','news_final_date'])

users_news = pd.merge(users_news, noticias[['news_id', 'news_title', 'news_text_content']], on='news_id').sort_values(by=['nit']).reset_index(drop=True)
for col in users_news.columns[2:]:
  users_news[col] = users_news[col].str.lower()
  if col != 'news_url_absolute': 
    users_news[col] = users_news[col].replace(r"[^A-Za-z\s]","", regex=True)
    users_news[col] = users_news[col].replace(r"enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre|lunes|martes|miercoles|jueves|viernes|sabado|domingo", "", regex=True)
    
file = users_news.to_csv('/content/drive/MyDrive/Bancolombia_Dataton_2022/data_set.csv')

In [ ]:
users_news = pd.read_csv('data_set.csv', index_col=[0])
users_news

,nit,news_id,news_url_absolute,nombre,subsec,news_title,news_text_content
0,800000946,news15986,https://www.infobae.com/cultura/agenda-cultura...,procter gamble colombia ltda,manufactura de productos de tocador y articulo...,guia de arte y cultura semana del al de,arte agustina galindez quesada exhibe catarsis...
1,800000946,news89860,https://acento.com.do/economia/wall-street-cie...,procter gamble colombia ltda,manufactura de productos de tocador y articulo...,wall street cierra el en verde,wall street cerro en verde este y el dow jone...
2,800000946,news89859,https://www.semana.com/economia/inversionistas...,procter gamble colombia ltda,manufactura de productos de tocador y articulo...,wall street termina al alza su mejor mes del a...,la bolsa de nueva york finalizo con nota posit...
3,800000946,news89850,https://sportingcristal.org/page/9954/,procter gamble colombia ltda,manufactura de productos de tocador y articulo...,pagina,el informe ofrece los desarrollos regionales d...
4,800000946,news69546,https://acento.com.do/economia/wall-street-cie...,procter gamble colombia ltda,manufactura de productos de tocador y articulo...,wall street cierra a la baja este y el dow jo...,wall street cerro este en rojo y el dow jones...
...,...,...,...,...,...,...,...
74612,901576411,news96699,https://www.infobae.com/america/eeuu/2022/07/1...,mauria group sas,hoteles,tres hoteles del sur de la florida entre los ...,como todos los anos la popular publicacion dig...
74613,901576411,news96696,https://www.eltiempo.com/deportes/futbol-inter...,mauria group sas,hoteles,catar en el paredon denuncian acoso sexual y r...,hola aqui tambien puedes encontrar mis noticia...
74614,901576411,news96693,https://www.larepublica.co/empresas/hotel-este...,mauria group sas,hoteles,hotel estelar registro ocupacion de durante e...,en su primer semestre de operacion el hotel es...
74615,901576411,news98685,https://www.rcnradio.com/economia/el-desempleo...,mauria group sas,hoteles,el desempleo en colombia sigue cayendo en lle...,el departamento nacional de estadisticas dane ...


In [ ]:
users_news.nombre.unique().shape, users_news.news_id.unique().shape

((1506,), (23338,))

**Nota**:

 39 noticias no tienen asociado un cliente y hay 1 cliente que no tiene noticias asociadas. Por eso hay menos nit´s y menos noticias únicos en este dataframe. 
 
  A partir del data Frame almacenado anteriormente, se genera un data set de noticias únicas (`noticias_unicas.csv`) el cual será utilizado en siguientes pasos.

In [ ]:
noticias_unicas = users_news[["news_id"]]
noticias_unicas["news_text_content"] = users_news.news_title +  " " + users_news.news_text_content

noticias_unicas.drop_duplicates(inplace=True, ignore_index=True)

<ipython-input-9-aecc6849b487>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noticias_unicas["news_text_content"] = users_news.news_title +  " " + users_news.news_text_content
<ipython-input-9-aecc6849b487>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noticias_unicas.drop_duplicates(inplace=True, ignore_index=True)


Con este dataset de noticias únicas se implementará el método de categorización (LDA) como se verá acontinuación.

# Stopwords

Para la lectura ágil y requerida por el método a implementar, suele ser de gran utilidad eliminar todas aquellas palabras que no contengan información de relevancia (ej. artículos (el, la, los, ...). Se utiliza las stop_words en español de Spacy que tiene alrededor de 500 palabras. Además, se quitan todas las tildes y las ñ's de los stop words, dado que en el Scraping de la noticias se eliminaron.

In [ ]:
!pip install spacy

In [ ]:
import spacy
from spacy.cli import download
print(download('es_core_news_sm'))
nlp = spacy.load('es_core_news_sm')
spacy_stopwords_es = spacy.lang.es.stop_words.STOP_WORDS
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ñ", "n")
    )
    for a, b in replacements:
        s = s.replace(a, b)
    return s
stop_words = [normalize(word) for word in spacy_stopwords_es]

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
None


**Nota:** Ideas (alternativas a tener en cuenta)
* Analisis de pesos personalizado por cliente, concatenar todas las noticias asociadas a un solo cliente y hacer un LDA o Hugging Face, y de la distribución de probabilidad producida generar la importancia a partir de pesos de cada categoria o tema. No se implementó dado el costo en tiempo computacional requerido para el modelamiento de tópicos con Latent Dirichlet Allocation a cada cliente único, alrededor de 1500 LDA's diferentes. 

# Categorización

## Latent Dirichlet Allocation. Topic Modelling. LDA SKlearn

#### Distribucíon de tópicos de todas las noticias en un sistema métrico. 

Se hace un LDA con todas las noticias únicas, luego, con los keywords asociados a cada tópico numérico se hace un Hugging Face concatenándolos y asignando el tópico con mayor probabilidad al número en cuestión. 

Se decidió usar las noticias únicas dado que es un data set con menor cantidad de datos que el data set que incluye las noticias con su respectivo cliente, es una relación de aproximadamente 23000 a 74000 noticias respectivamente. Además, para la categorización de las noticias (identificación de tópico) no es necesario tener en cuenta al cliente, pues la noticia hablará de un tema en específico independientemente del cliente. Una vez realizada la categorización a cada noticia se redistribuirán a sus respectivos clientes con su tema identificado.  

In [ ]:
#importacion de librerias a usar en LDA
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer  
from nltk.tokenize import  word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Como se mencionó, para el preporcesamineto de texto, eliminamos las stop words de las noticias y algunas frases repetidas en una gran cantidad de noticias debidas a fallas en el Scraping, pues estas frases tienen que ver con publicidad o cookies de la página web. Además, se eliminan todas las palabras con 3 caracteres o menos, pues hay presencia de muchas palabras en ingles como the, is, etc.

In [ ]:
def clean_text(headline):
      le=WordNetLemmatizer()
      word_tokens=word_tokenize(str(headline))
      tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
      cleaned_text=" ".join(tokens)
      return cleaned_text
noticias_unicas['news_text_content_cleaned'] = noticias_unicas['news_text_content'].apply(clean_text)
noticias_unicas["news_text_content_cleaned"] = noticias_unicas["news_text_content_cleaned"].replace(r"marcar autorizo envio newsletter avisos informativos personalizados interempresasnet autorizo envio comunicaciones terceros interempresasnet leido acepto aviso legal politica proteccion datos","", regex=True)
noticias_unicas["news_text_content_cleaned"] = noticias_unicas["news_text_content_cleaned"].replace(r"proyectos latinoamerica empresas mundo negocios region contactos clave relacionados empresas proyectos analisis reportajes noticias entrevistas industria ingles espanol portugues","", regex=True)

<ipython-input-16-b393054c6d2c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noticias_unicas['news_text_content_cleaned'] = noticias_unicas['news_text_content'].apply(clean_text)
<ipython-input-16-b393054c6d2c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noticias_unicas["news_text_content_cleaned"] = noticias_unicas["news_text_content_cleaned"].replace(r"marcar autorizo envio newsletter avisos informativos personalizados interempresasnet autorizo envio comunicaciones terceros interempresasnet leido

TFIDF vectorization on the text column.
Vectores Term Frequency-Inverse Document Frequency (TF-IDF) para cada noticia. Esto producirá una matriz donde cada columna representa una palabra en el vocabulario general (todas las palabras que aparecen en al menos una noticia).

In [ ]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000)
vect_text=vect.fit_transform(noticias_unicas["news_text_content_cleaned"])

Implementamos LDA sobre el texto vectorizado, entrenamos el modelo y transformamos el texto vectorizado. Se ingresa como argumento de LDA 20 componentes, es decir, dividir las noticias en 20 tópicos. 

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model=LatentDirichletAllocation(n_components=20,
          learning_method='online',random_state=0,max_iter=10) #online es mas rapido que batch cuando el tamaño de datos es mas grande
lda_top=lda_model.fit_transform(vect_text)

Checking the results, probamos el modelo con una noticia y damos un vistazo a la distribución de probabilidad.

In [ ]:
print("news 0: ")
for i,topic in enumerate(lda_top[1]):
  print("Topic ",i+1,": ",topic*100,"%")

news 0: 
Topic  1 :  85.57131616689999 %
Topic  2 :  0.6216805541581178 %
Topic  3 :  0.6216805604668376 %
Topic  4 :  3.238421620010379 %
Topic  5 :  0.6216896287293169 %
Topic  6 :  0.621680554360188 %
Topic  7 :  0.6216805541581182 %
Topic  8 :  0.6216806935752798 %
Topic  9 :  0.6216805541581192 %
Topic  10 :  0.6216805541581175 %
Topic  11 :  0.6216805581934927 %
Topic  12 :  0.621680554158119 %
Topic  13 :  0.6216811483664518 %
Topic  14 :  0.6216805830295653 %
Topic  15 :  0.6216805647226009 %
Topic  16 :  0.6216824190343073 %
Topic  17 :  0.6216808693189771 %
Topic  18 :  0.6216805849113184 %
Topic  19 :  0.6216805608994135 %
Topic  20 :  0.6216807166912851 %


Analyzing the Topics, se imprimen las 20 primeras keywords de los 20 tópicos en orden de mayor a menor frecuencia presentada en las noticias pertenecientes a dicho tópico. 

In [ ]:
vocab = vect.get_feature_names()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("\nTop 20 words in Topic "+str(i+1)+": ")
     for t in sorted_words:
            print(t[0],end=" ")
     print("\n")
            


Top 20 words in Topic 1: 
millones mercado inflacion precios dolares dolar precio banco aumento peso crecimiento tasa economia trimestre colombia pais empresas tasas unidos valor 


Top 20 words in Topic 2: 
vino argentina registro edicion acceso contenidos responsable propiedad feria medellin exportaciones dolar viruela naciones plantas aire francia joven valle imagen 


Top 20 words in Topic 3: 
venezuela peru brasil relaciones comercial eeuu edificio apertura paises taiwan unido agenda colombia plantas instalaciones emisiones iniciativa lideres pedro secretario 


Top 20 words in Topic 4: 
millones euro china taiwan billones credito inversion capital fondo inversiones fondos cartera grupo financiamiento banco semestre espana operaciones resultados operacion 


Top 20 words in Topic 5: 
salud hospital pacientes enfermedad viruela medico casos covid tratamiento atencion enfermedades persona centro prevencion publica contacto pruebas emergencia poblacion secretaria 


Top 20 words in 

In [ ]:
!pip install pyldavis   

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136904 sha256=487978d170cce79f95765c589c4fdc27a6dde5129415f345ab6a1dd508f164ae
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\90\61\ec\9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2343 sha256=9091e1678c48d0232c62cc5a4d89be129eb90dd3cacaef92ba6674ace4d53abf
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\14\25\f7\1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully bui

Se usa la librería pyLDAvis, muy útil para visualizar la división de tópicos realizada por LDA en una representación vectorial de dimensión reducida 2-dimensional basada en distancias t-distributed Stochastic Neighbor Embedding. 

In [ ]:
import pyLDAvis 
import pyLDAvis.sklearn 

pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_model, vect_text, vect, mds='tsne') #t-distributed Stochastic Neighbor Embedding

S:\Anaconda\lib\site-packages\sklearn\manifold\_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
16     -10.878068   10.538728       1        1  32.046604
7      -60.884026  -75.591820       2        1  20.170684
19      -2.336119  -53.289074       3        1  17.508670
0      -61.385319  -12.914634       4        1  11.186976
18     -28.737743  126.296997       5        1   3.104765
13       1.066606 -123.552811       6        1   1.803639
15      45.140720   -7.947869       7        1   1.588416
14    -140.795975  -98.530548       8        1   1.385609
3       61.531681  -83.240562       9        1   1.261315
10     -72.563057 -140.396881      10        1   1.247032
4       15.984491   63.578186      11        1   1.212352
5     -160.851624   68.945145      12        1   1.043999
12      85.961830   51.922935      13        1   0.949753
17     -99.020836  109.936348      14        1   0.938678
9     -176.764130   -8.681684      15        1   0.817476
2     -117.189880  -37.420200      16        1   0.775715
1      108.829918  -22.946203      17        1   0.768635
6       47.588097  121.934814      18        1   0.748196
11    -106.467834   30.176889      19        1   0.721303
8      -50.613804   59.328842      20        1   0.720183, topic_info=          Term        Freq       Total Category  logprob  loglift
213      cooky  724.000000  724.000000  Default  30.0000  30.0000
209   contrato  494.000000  494.000000  Default  29.0000  29.0000
102       cafe  483.000000  483.000000  Default  28.0000  28.0000
604     mexico  523.000000  523.000000  Default  27.0000  27.0000
793   proyecto  573.000000  573.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
745  practicas    0.516839   86.297795  Topic20  -7.5906  -0.1844
784   promover    0.516839   88.546765  Topic20  -7.5906  -0.2101
53       apoyo    0.516839  185.907433  Topic20  -7.5906  -0.9519
720      petro    0.516839  469.815948  Topic20  -7.5906  -1.8789
979    version    0.516839  116.133814  Topic20  -7.5906  -0.4813

[920 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1         1  0.415913  acceso
1         2  0.110095  acceso
1         3  0.085629  acceso
1         4  0.053009  acceso
1        17  0.326206  acceso
...     ...       ...     ...
999       3  0.096023    zona
999       4  0.051441    zona
999       5  0.020576    zona
999       6  0.013718    zona
999      20  0.003429    zona

[3231 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 8, 20, 1, 19, 14, 16, 15, 4, 11, 5, 6, 13, 18, 10, 3, 2, 7, 12, 9])

Se añade la columna que contiene la categoría o tema más relacionado a cada noticia al dataframe. La lista de tópicos se ha creado leyendo las palabras clave y relacionándolas con uno de los tópicos, para los tópicos que no se sabía o no era tan claro el tema adecuado. Con ayuda de Hugging Face se pasó la lista de palabras concatenadas como string junto con los tópicos de Macroeconomía, Sostenibilidad, innovación, etc., y con la distribución de probabilidad resultante por Hugging Face se le otorgó la de mayor probabilidad. 

In [ ]:
topicos=["macroeconomia", "otra", "alianzas", "macroeconomia", "descartable", "descartable", "descartable",
         "regulaciones", "macroeconomia", "alianzas", "reputacion", "innovacion", "regulaciones", 
         "sostenibilidad", "reputacion", "regulaciones", "innovacion", "sostenibilidad", "descartable", "regulaciones"]
topic_new = []         
for i in range(len(lda_top)):
  topic_number = np.argsort(lda_top[i])[::-1][0]
  topic_new.append(topicos[topic_number])

noticias_unicas["category"] = np.r_[topic_new]

S:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-22-7c7d0761f435>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noticias_unicas["category"] = np.r_[topic_new]


Se observa cómo quedaron distribuídos los temas a lo largo de todas las noticias únicas del dataset:

In [ ]:
noticias_unicas.groupby(by="category").count()

S:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,news_id,news_text_content,news_text_content_cleaned
category,,,
alianzas,64,64,64
descartable,1287,1287,1287
innovacion,7784,7784,7784
macroeconomia,3078,3078,3078
otra,31,31,31
regulaciones,10524,10523,10524
reputacion,297,297,297
sostenibilidad,273,273,273


Finalmente guardamos los resultados en un .csv para continuar sobre este las próximas sesiones.

In [ ]:
noticias_unicas.to_csv("category.csv")
noticias_unicas

S:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,news_id,news_text_content,news_text_content_cleaned,category
0,news15986,guia de arte y cultura semana del al de art...,guia arte cultura semana arte agustina galinde...,regulaciones
1,news89860,wall street cierra el en verde wall street ce...,wall street cierra verde wall street cerro ver...,macroeconomia
2,news89859,wall street termina al alza su mejor mes del a...,wall street termina alza pese disparada inflac...,macroeconomia
3,news89850,pagina el informe ofrece los desarrollos regi...,pagina informe ofrece desarrollos regionales m...,innovacion
4,news69546,wall street cierra a la baja este y el dow jo...,wall street cierra baja jones pierde wall stre...,macroeconomia
...,...,...,...,...
23333,news79504,amazon comprara a fabricante de aspiradoras ro...,amazon comprara fabricante aspiradoras roomba ...,macroeconomia
23334,news79523,pokemon company presento todas las novedades d...,pokemon company presento novedades takato utsu...,regulaciones
23335,news79502,el proyecto de ley sobre el clima de ee uu pod...,proyecto clima reducir degc calentamiento apro...,innovacion
23336,news79511,aduanas certifica empresas como operadores ec...,aduanas certifica empresas operadores economic...,innovacion


# Participación

Para identificar si el cliente participa directamente en la noticia, su sector se menciona en esta o ninguna de las anteriores, procedemos de la siguiente manera.

Se crean cinco funciones:
* `remove_stopwords`   que se encargan de remover los stop words y eliminar todas las letras sueltas que queden en el texto de la noticia.
* `lemmatizer` lemmatiza el texto ingresado.
* `tokenize` tokenizaa el texto ingresado convirtiendolo en una lista con las palabras que contenía el texto. 

Dos identificadores:
* `is_sector` verifica si dada una lista con palabras alguna de estas se encuentra en el texto ingresado para saber si el sector del cliente se menciona en el texto.
* `is_client` verifica si dado un nombre de cliente y un texto este se encuentra en él.

La implementación de todas estas funcionalidades se muestran en la celda siguiente:

In [ ]:
def remove_stopwords(headline):
  word_tokens=word_tokenize(str(headline))
  tokens=[w for w in word_tokens if w not in stop_words and len(w)>1]
  cleaned_text=" ".join(tokens)
  return cleaned_text

def lemmatizer(text_):
  text = remove_stopwords(text_)
  nlp = spacy.load("es_core_news_sm", disable=["parser", "tagger", "ner"])
  nlp.max_length = 2_000_000

  if len(text)<500000:
    doc = nlp(text)
    empty_list = []
    for token in doc:
        empty_list.append(normalize(token.lemma_))
    text1 = ' '.join(map(str,empty_list))
    return text1
  
  else: 
    return text

def tokenize(headline_):
  #headline = lemmatizer(headline_)
  word_tokens=word_tokenize(str(headline_))
  tokens=[w for w in word_tokens]
  return np.unique(tokens)

def is_sector(tokens, text):
  #text = lemmatizer(text_)
  return any(re.search(r"\b" + re.escape(word) + r"\b", text) for word in tokens)

def is_client(name, text):
  value = False
  if re.findall(name, text): 
    value = True
  return value

Se lematizan todas las noticias únicas del dataset (**NOTA IMPORTANTE**: este proceso tomó alrededor de **3 horas**, por esto se recomienda no volver a ejecutar) y se guardó un .csv con el resultado.

**Nota adicional**
Debido a la eliminación de acentos en los datos suministrados, la lematización no puede ser precisa. Para resultados más "limpios" se recomienda texto con acentos. Recordemos que en el idioma Español es de gran relevancia los acentos.

In [ ]:
noticias_unicas = pd.read_csv('category.csv', index_col=[0])
noticias_unicas.news_text_content = noticias_unicas.news_text_content.astype(str, copy=False).apply(lemmatizer)

In [ ]:
noticias_unicas.to_csv('noticias_lemma.csv') #3 horas

Una vez lemmatizadas todas las noticias, se lee el archivo y se escogen las columnas a usar concernientes a la participación del cliente en la noticia.

In [ ]:
noticias_unicas = pd.read_csv('noticias_lemma.csv', index_col=[0]) #carga de noticias lemmatizadas
users_news = pd.read_csv('data_set.csv', index_col=[0])

In [ ]:
users_news[["news_id", "news_title", "news_text_content"]]

,news_id,news_title,news_text_content
0,news15986,guia de arte y cultura semana del al de,arte agustina galindez quesada exhibe catarsis...
1,news89860,wall street cierra el en verde,wall street cerro en verde este y el dow jone...
2,news89859,wall street termina al alza su mejor mes del a...,la bolsa de nueva york finalizo con nota posit...
3,news89850,pagina,el informe ofrece los desarrollos regionales d...
4,news69546,wall street cierra a la baja este y el dow jo...,wall street cerro este en rojo y el dow jones...
...,...,...,...
74612,news96699,tres hoteles del sur de la florida entre los ...,como todos los anos la popular publicacion dig...
74613,news96696,catar en el paredon denuncian acoso sexual y r...,hola aqui tambien puedes encontrar mis noticia...
74614,news96693,hotel estelar registro ocupacion de durante e...,en su primer semestre de operacion el hotel es...
74615,news98685,el desempleo en colombia sigue cayendo en lle...,el departamento nacional de estadisticas dane ...


Se crea otra columna en el dataset anterior donde las noticias no están lemmatizadas, pero sí se han eliminado las stop words y letras sueltas.

In [ ]:
noticias = users_news[["news_id", "news_title", "news_text_content"]]
noticias.drop_duplicates(inplace=True, ignore_index=True)
noticias_unicas["news_text_content_cleaned"] = noticias.news_title + " " + noticias.news_text_content
noticias_unicas["news_text_content_cleaned"] = noticias_unicas["news_text_content_cleaned"].apply(lambda x: remove_stopwords(str(x)))

<ipython-input-48-03c979a261a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noticias.drop_duplicates(inplace=True, ignore_index=True)


In [ ]:
noticias_unicas

,news_id,news_text_content,news_text_content_cleaned,category
0,news15986,guia arte cultura seman arte agustin galindez ...,guia arte cultura semana arte agustina galinde...,regulaciones
1,news89860,wall street cierra verde wall street cerro ver...,wall street cierra verde wall street cerro ver...,macroeconomia
2,news89859,wall street terminar alza mes ano pese dispara...,wall street termina alza mes ano pese disparad...,macroeconomia
3,news89850,pagina informe ofrecer desarrollo regional mer...,pagina informe ofrece desarrollos regionales m...,innovacion
4,news69546,wall street cerrar bajo dow jones perder wall ...,wall street cierra baja dow jones pierde wall ...,macroeconomia
...,...,...,...,...
23333,news79504,amazon comprar fabricante aspiradora roomba ir...,amazon comprara fabricante aspiradoras roomba ...,macroeconomia
23334,news79523,pokemon company presento novedad takatir utsun...,pokemon company presento novedades takato utsu...,regulaciones
23335,news79502,proyecto ley climo ee uu reducir degc calentam...,proyecto ley clima ee uu reducir degc calentam...,innovacion
23336,news79511,aduana certificar empresa operador economico a...,aduanas certifica empresas operadores economic...,innovacion


Se fusiona este dataset anterior (`noticias_unicas`) con el dataset donde aparecen los clientes con sus respectivas noticas lematizadas y no lematizadas, etc. (`users_news`) para crear el dataset (`participacion`) que es el que contendrá la información suficiente para llevar a cabo la participación de cada cliente en la noticia.

In [ ]:
users_news.drop(columns=['news_text_content'], inplace=True)
participacion = pd.merge(users_news, noticias_unicas[['news_id', 'category', 'news_text_content', 'news_text_content_cleaned']], on='news_id')
clientes = pd.read_csv('clientes.csv')


Con el dataset (`clientes`) que tiene la información de los clientes extraemos la información concerniente al `subsector`, y `códigos ciiu división`, `ciuu grupo` y `ciiuu clase`; además, se almacena concatenándolos como un string, pues estas palabras serán los tokens usados sobre:
* las noticias **lemmatizadas** para verificar que el **sector** del cliente aparece en la noticia,
* y el nombre del **cliente** será usado sobre las noticias **sin lemmatizar** para verificar la participación directa en la noticia.

Ademas se lemmatiza este string concatenado mencionado anteriormente (_se demora aproximadamente 30 min. en compilar_), lo almacenamos en el dataset (`clientes`) y lo guardamos como otro .csv  

In [ ]:
clientes["participate_sectors"] = clientes["subsec"].str.lower()+" "+clientes["desc_ciiu_division"].str.lower()+ " " + clientes["desc_ciuu_grupo"].str.lower()+ " " + clientes["desc_ciiuu_clase"].str.lower()
clientes["participate_sectors"] = clientes["participate_sectors"].apply(lemmatizer)

S:\Anaconda\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'tagger', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [ ]:
clientes.to_csv('clientes_lemma.csv')

Se lee el .csv con los clientes lemmatizados.

In [ ]:
clientes = pd.read_csv('clientes_lemma.csv', index_col=[0])

  Finalmente, fusionamos la información del string concatenado almacenada en el dataset (clientes) con el dataset (participacion), anexamos al texto de la noticia lemmatizada la URL del sitio web de la noticia porque en ocasiones dentro del mismo link aparece el sector del que habla la noticia, eliminamos las columnas que no se usarán, se eliminan los tipos de sociedades colombianas como SAS, SA, LTD, etc. de los nombres de los clientes y los espacios múltiples que hayan quedado.

In [ ]:
participacion = pd.merge(participacion, clientes[['nit', 'participate_sectors']], on='nit')
participacion["news_text"] = (participacion.news_url_absolute  +  " " + participacion.news_text_content)

#for col in participacion.columns[5:]:
#  participacion[col] = participacion[col].str.lower()
participacion.drop(columns=["news_url_absolute", "subsec", "news_title", "news_text_content"], inplace=True)
participacion["nombre"] = participacion["nombre"].apply(lambda x: re.sub("\\sinc\\b|\\ssas\\b|\\ssa\\b|\\sltda?\\b|[^\sa-z]", "",x.lower()))
participacion["nombre"] = participacion["nombre"].apply(lambda x: re.sub("\s{2,}", " ",x.lower()))


In [ ]:
participacion.head(2)

,nit,news_id,nombre,category,news_text_content_cleaned,participate_sectors,news_text
0,800000946,news15986,procter gamble colombia,regulaciones,guia arte cultura semana arte agustina galinde...,manufactura producto tocador articulo aseo fab...,https://www.infobae.com/cultura/agenda-cultura...
1,800000946,news89860,procter gamble colombia,macroeconomia,wall street cierra verde wall street cerro ver...,manufactura producto tocador articulo aseo fab...,https://acento.com.do/economia/wall-street-cie...


Con un ciclo `for` generamos el array (`roles`) que contiene la participación de cada cliente. **Recordemos** que para la identificar si el cliente está en la noticia se usa la noticia sin lemmatizar y para identificar si es el sector quien participa se usa la noticia lemmatizada.

In [ ]:
roles = []
for i in range(len(participacion)):
  rol = "no_aplica"
  row = participacion.iloc[i]
  tokens = tokenize(row.participate_sectors)

  if is_client(row.nombre, str(row.news_text_content_cleaned)):
    rol = "cliente"
  elif is_sector(tokens, str(row.news_text)):
    rol = "sector"
  roles.append(rol)

participacion["participacion"] = np.r_[roles]  

Se muestra el dataset (`participacion`) resultante

In [ ]:
participacion

,nit,news_id,nombre,category,news_text_content_cleaned,participate_sectors,news_text,participacion
0,800000946,news15986,procter gamble colombia,regulaciones,guia arte cultura semana arte agustina galinde...,manufactura producto tocador articulo aseo fab...,https://www.infobae.com/cultura/agenda-cultura...,sector
1,800000946,news89860,procter gamble colombia,macroeconomia,wall street cierra verde wall street cerro ver...,manufactura producto tocador articulo aseo fab...,https://acento.com.do/economia/wall-street-cie...,sector
2,800000946,news89859,procter gamble colombia,macroeconomia,wall street termina alza mes ano pese disparad...,manufactura producto tocador articulo aseo fab...,https://www.semana.com/economia/inversionistas...,sector
3,800000946,news89850,procter gamble colombia,innovacion,pagina informe ofrece desarrollos regionales m...,manufactura producto tocador articulo aseo fab...,https://sportingcristal.org/page/9954/ pagina ...,sector
4,800000946,news69546,procter gamble colombia,macroeconomia,wall street cierra baja dow jones pierde wall ...,manufactura producto tocador articulo aseo fab...,https://acento.com.do/economia/wall-street-cie...,no_aplica
...,...,...,...,...,...,...,...,...
74612,900178724,news26364,medplus medicina prepagada,innovacion,adultos colombia contaba superintendencia fina...,eps salud prepagado salud seguro reaseguro seg...,https://m.vanguardia.com/economia/nacional/9-d...,sector
74613,900178724,news98977,medplus medicina prepagada,innovacion,penetracion seguros mantuvo alza primas inform...,eps salud prepagado salud seguro reaseguro seg...,https://www.larepublica.co/finanzas/la-penetra...,sector
74614,900178724,news98981,medplus medicina prepagada,regulaciones,responde colombia gobierno problemas adquirir ...,eps salud prepagado salud seguro reaseguro seg...,https://publimotos.com/19-mundo/colombia/5929-...,sector
74615,900178724,news59181,medplus medicina prepagada,innovacion,axa colpatria comprometida inversiones verdes ...,eps salud prepagado salud seguro reaseguro seg...,https://www.larepublica.co/especiales/seguros-...,sector


Se muestra **la distribución de participacion** por cliente, sector y no_aplica a lo largo de todo el dataset (`participacion`).

In [ ]:
participacion.groupby(by="participacion").count()

,nit,news_id,nombre,category,news_text_content_cleaned,participate_sectors,news_text
participacion,,,,,,,
cliente,2790,2790,2790,2790,2790,2790,2790
no_aplica,17887,17887,17887,17887,17887,17887,17887
sector,53940,53940,53940,53940,53940,53940,53940


Se añade una columna con el nombre del equipo y se eliminan las columnas innecesarias. Se guarda el .csv.

**Esto corresponde al primer entregable**.

In [ ]:
participacion["nombre_equipo"] = "Physicist.py"
participacion_category = participacion[["nombre_equipo", "nit", "news_id", "participacion", "category"]]
participacion_category.rename(columns = {'category':'categoria'}, inplace = True)
participacion_category.to_csv('categorizacion.csv')
participacion_category

S:\Anaconda\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,nombre_equipo,nit,news_id,participacion,categoria
0,Physicist.py,800000946,news15986,sector,regulaciones
1,Physicist.py,800000946,news89860,sector,macroeconomia
2,Physicist.py,800000946,news89859,sector,macroeconomia
3,Physicist.py,800000946,news89850,sector,innovacion
4,Physicist.py,800000946,news69546,no_aplica,macroeconomia
...,...,...,...,...,...
74612,Physicist.py,900178724,news26364,sector,innovacion
74613,Physicist.py,900178724,news98977,sector,innovacion
74614,Physicist.py,900178724,news98981,sector,regulaciones
74615,Physicist.py,900178724,news59181,sector,innovacion


# Recomendación

Para la recomendación se usa un sistema de pesos de acuerdo al tópico al que pertenece la noticia y de a cuerdo a la participación que tiene el cliente. De manera que:

* Categorización:

  * `macroeconomia`: 2
  * `sostenibilidad`: 1.8
  * `innovacion`: 1.7
  * `regulaciones`: 1.6
  * `alianzas`: 1.4
  * `reputacion`: 1.2
  * `otra`: 0.5
  * `descartable`: 0.3

* `Participación`:

  * `cliente`: 2
  * `sector`: 1.2
  * `no_aplica`: 0.3

Se empieza cargando los datos que contienen el primer entregable

In [ ]:
participacion_category = pd.read_csv('categorizacion.csv', index_col=[0]) 
participacion_category

,nombre_equipo,nit,news_id,participacion,categoria
0,Physicist.py,800000946,news15986,sector,regulaciones
1,Physicist.py,800000946,news89860,sector,macroeconomia
2,Physicist.py,800000946,news89859,sector,macroeconomia
3,Physicist.py,800000946,news89850,sector,innovacion
4,Physicist.py,800000946,news69546,no_aplica,macroeconomia
...,...,...,...,...,...
74612,Physicist.py,900178724,news26364,sector,innovacion
74613,Physicist.py,900178724,news98977,sector,innovacion
74614,Physicist.py,900178724,news98981,sector,regulaciones
74615,Physicist.py,900178724,news59181,sector,innovacion


Creamos la función (`Recomendacion`) que le entra como argumento un string con la `participación` y otro string con la `categoria` de la noticia en cuestión, luego se hace un diccionario con los pesos establecidos y retorna la multiplicación de los pesos correspondientes. 

In [ ]:
def Recomendacion(participacion, categoria):
    labels = ["macroeconomia","sostenibilidad","innovacion", "regulaciones", "alianzas", "reputacion", "otra", "descartable",
             "cliente", "sector", "no_aplica"] 
    weighs = [2, 1.8, 1.7, 1.6, 1.4, 1.2, 0.5, 0.3, 2, 1.2, 0.3]
    dicc = dict(zip(labels,weighs))

    return dicc[categoria]*dicc[participacion]

participacion_category["recomendacion"] = participacion_category.apply(lambda x: Recomendacion(x.participacion, 
                                                                                               x.categoria), axis=1)
participacion_category

,nombre_equipo,nit,news_id,participacion,categoria,recomendacion
0,Physicist.py,800000946,news15986,sector,regulaciones,1.92
1,Physicist.py,800000946,news89860,sector,macroeconomia,2.40
2,Physicist.py,800000946,news89859,sector,macroeconomia,2.40
3,Physicist.py,800000946,news89850,sector,innovacion,2.04
4,Physicist.py,800000946,news69546,no_aplica,macroeconomia,0.60
...,...,...,...,...,...,...
74612,Physicist.py,900178724,news26364,sector,innovacion,2.04
74613,Physicist.py,900178724,news98977,sector,innovacion,2.04
74614,Physicist.py,900178724,news98981,sector,regulaciones,1.92
74615,Physicist.py,900178724,news59181,sector,innovacion,2.04


Se ordena el dataset (`participacion_category`) teniendo en cuenta dos columnas de la siguiente manera: ordenando de menor a mayor la identificación del cliente (`nit`) y luego para cada nit ordenando de mayor a menor la recomendación. Este nuevo dataframe lo llamamos (`recomendacion_rank`).

In [ ]:
#Ordena en cada nit de tal manera que la recomendacion quede de mayor a menor
recomendacion_rank = participacion_category.sort_values(by=["nit", "recomendacion"], ascending=[True,False])
recomendacion_rank

,nombre_equipo,nit,news_id,participacion,categoria,recomendacion
1,Physicist.py,800000946,news89860,sector,macroeconomia,2.40
2,Physicist.py,800000946,news89859,sector,macroeconomia,2.40
5,Physicist.py,800000946,news97858,sector,macroeconomia,2.40
11,Physicist.py,800000946,news97669,sector,macroeconomia,2.40
3,Physicist.py,800000946,news89850,sector,innovacion,2.04
...,...,...,...,...,...,...
45999,Physicist.py,901576411,news79522,no_aplica,regulaciones,0.48
46001,Physicist.py,901576411,news79523,no_aplica,regulaciones,0.48
45981,Physicist.py,901576411,news29628,sector,descartable,0.36
45975,Physicist.py,901576411,news96710,no_aplica,descartable,0.09


Se crea la función que hace la jerarquización. 

Descripción:

* Le ingresa el dataframe (`recomendacion_rank`),
* se hace una copia de este para evitar problemas de referencia,
* se recorre sobre cada nit único, se crea un dataframe (`jerarq_client`) con solo esos nit's únicos,
* se sacan sus índices respectivos para ubicar las posiciones del dataframe (`recomendacion_rank`),
* se hace una lista (`scores_reco`) con los puntajes de recomendación únicos en el dataframe (`jerarq_client`), se recorre sobre cada índice, 
* modificamos la posición con ese índice en el dataframe (`recomendacion_rank`) con el ranking de noticias, este ranking de noticias resulta de encontrar qué posicion de la lista de puntajes (`scores_reco`) concuerda con el puntaje que tiene el índice donde estamos localizados, le suma 1 y el resultado es la posición en el ranking. 

Cabe aclarar que este sistema de pesos permite que puedan haber empates en la recomendación, por ejemplo 2 noticias que se recomienden leer en primer lugar. 

**Finalmente se guarda el** `recomendacion.csv` **con los resultados que son los correspondientes a la segunda entrega.** 





In [ ]:
#asocia el valor de la participacion al respectivo puesto en el ranking de recomendacion
def jerarquia(DataFrame_):
    DataFrame = DataFrame_.copy(deep=True)
    nit_ = DataFrame["nit"].unique()
    
    for nit in nit_:
        jerarq_client = DataFrame[DataFrame["nit"]==nit]
        key_ = jerarq_client.index
        scores_reco = jerarq_client.recomendacion.unique()
        for key in key_:
            DataFrame.loc[key,"recomendacion"] = int(np.where(DataFrame.loc[key,"recomendacion"] == scores_reco)[0][0]+1)
    
    return DataFrame

recomendacion_rank = jerarquia(recomendacion_rank)
recomendacion_rank.to_csv('recomendacion.csv')
recomendacion_rank

,nombre_equipo,nit,news_id,participacion,categoria,recomendacion
1,Physicist.py,800000946,news89860,sector,macroeconomia,1.0
2,Physicist.py,800000946,news89859,sector,macroeconomia,1.0
5,Physicist.py,800000946,news97858,sector,macroeconomia,1.0
11,Physicist.py,800000946,news97669,sector,macroeconomia,1.0
3,Physicist.py,800000946,news89850,sector,innovacion,2.0
...,...,...,...,...,...,...
45999,Physicist.py,901576411,news79522,no_aplica,regulaciones,6.0
46001,Physicist.py,901576411,news79523,no_aplica,regulaciones,6.0
45981,Physicist.py,901576411,news29628,sector,descartable,7.0
45975,Physicist.py,901576411,news96710,no_aplica,descartable,8.0


Se muestra la recomendación, categorizacón y participación de las noticias asociadas al primer cliente.  

In [ ]:
recomendacion_rank[:15]

,nombre_equipo,nit,news_id,participacion,categoria,recomendacion
1,Physicist.py,800000946,news89860,sector,macroeconomia,1.0
2,Physicist.py,800000946,news89859,sector,macroeconomia,1.0
5,Physicist.py,800000946,news97858,sector,macroeconomia,1.0
11,Physicist.py,800000946,news97669,sector,macroeconomia,1.0
3,Physicist.py,800000946,news89850,sector,innovacion,2.0
6,Physicist.py,800000946,news97853,sector,innovacion,2.0
7,Physicist.py,800000946,news97846,sector,innovacion,2.0
12,Physicist.py,800000946,news95244,sector,innovacion,2.0
15,Physicist.py,800000946,news97841,sector,innovacion,2.0
0,Physicist.py,800000946,news15986,sector,regulaciones,3.0
